In [199]:
import torch
import pandas as pd
import numpy as np

In [200]:
# retrieval - find relevant info - ask question abous an article -> retrieve passage of texts related to the query

# augmentation - take relevant info and augment input to LLM with relevant info

# generation - Take the first two steps and pass to LLM for GEN outs

## WHY RAG?

# the goal is to improve output of an LLM

# prevent hallucinations - the text is more factual not only good looking RAG can help LLMs generate information based on relevant passages that are factual
# work with custom data - not trained only with internet scale data like LLMs. However it also does a lot of the responses in a general nature. RAG helps to create specific responses based on specific documents

# use cases?

# customer support Q&A chat. Existing customer support docs as a resource, retrieval system based on that data, LLM crafts the answer based on those, basically a "chatbot for your docs".

# email chain analysis - chains of emails of claims. you can use rag pipeline to find relevant information from the chains and use LLM to process that info

# company internal documentation chat 

# textbook Q&A - build a rag pipeline to go through the textbook to find relevant passages to answer the questions

# Common theme - take documents to a query and process it with LLM, LLM -> kind of like a calculator for language

#### Text preprocessing

In [201]:
data = pd.read_csv("../data/medium.csv")
data.head()

Title  \
0          A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model   
1          Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric   
2                                             How to Use ggplot2 in Python   
3  Databricks: How to Save Data Frames as CSV Files on Your Local Computer   
4    A Step-by-Step Implementation of Gradient Descent and Backpropagation   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [202]:
def preprocess_text(data: pd.DataFrame) -> pd.DataFrame:
    data["Text"] = data["Text"].str.replace("\n", " ")
    data["Text"] = data["Text"].str.replace("\t", " ")
    data["Text"] = data["Text"].str.replace("\r", " ")
    data["Text"] = data["Text"].str.replace(r"\s+", " ", regex=True)
    data["Text"] = data["Text"].str.strip()

    return data


def sample_random_texts(data: pd.DataFrame, n=5) -> None:
    for text in data.sample(n)["Text"].values:
        print(text)
        print("\n\n")
        
        

In [203]:
data = preprocess_text(data)
data.head()

Title  \
0          A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model   
1          Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric   
2                                             How to Use ggplot2 in Python   
3  Databricks: How to Save Data Frames as CSV Files on Your Local Computer   
4    A Step-by-Step Implementation of Gradient Descent and Backpropagation   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [204]:
!python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### Feature engineering for text data EDA

In [205]:
from spacy.lang.en import English
from tqdm import tqdm
tqdm.pandas()

nlp = English()
nlp.add_pipe("sentencizer")

data["text_words_num"] = data["Text"].apply(lambda x: len(x.split()))
data["text_length"] = data["Text"].apply(lambda x: len(x))
data["text_sentences_num"] = data["Text"].progress_apply(lambda x: len(list(nlp(x).sents)))
data["token_count"] = data["Text"].apply(lambda x: len(x) / 4)
data.head()

100%|██████████| 1391/1391 [00:04<00:00, 326.70it/s]


Title  \
0          A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model   
1          Hands-on Graph Neural Networks with PyTorch & PyTorch Geometric   
2                                             How to Use ggplot2 in Python   
3  Databricks: How to Save Data Frames as CSV Files on Your Local Computer   
4    A Step-by-Step Implementation of Gradient Descent and Backpropagation   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [206]:
data.describe().round(2)

,text_words_num,text_length,text_sentences_num,token_count
count,1391.00,1391.00,1391.00,1391.00
mean,901.54,5530.81,44.46,1382.70
std,885.73,5521.48,44.17,1380.37
min,49.00,249.00,2.00,62.25
25%,315.00,1886.00,15.00,471.50
50%,516.00,3040.00,26.00,760.00
75%,1227.00,7577.50,60.00,1894.38
max,7657.00,46966.00,376.00,11741.50


#### Chunking articles into smaller sentences

In [207]:
for _, row in data.sample(5).iterrows():
    doc = nlp(row["Text"])
    sents = [sent.text for sent in doc.sents]
    print(sents)
    print("\n\n")

['Let’s create a micro-drone swarm that utilizes artificial intelligence and mesh networking to mimic swarm behavior that is typically seen in nature.', 'Hold on!', 'Don’t go!', 'Hear me out with this concept.', 'By utilizing these buzzwordy technologies we can create something pretty cool and dare I say, useful?', 'Sure, small drones have terrible flight times (among other things) and yes microprocessors might not be the best solution for AI processing.', 'However, if we keep the scope of the project small then we may be able to pull it off.', 'Something such as mimicking swarm behavior in an indoor environment with multiple quadcopters equipped with an array of sensors.', 'Still think I’m full of it?', 'What if I told you that I already have the first prototype built?', 'Let’s jump into it!', 'How Are The Drones Made?', 'When starting any project that requires hardware the first thing you should do is determine what components you want to use.', 'For this case, I opted to use a PCB f

In [208]:
n_sentences_in_chunk = 10
CONTEXT_WINDOW = 384


def split_text_into_chunks(text: str, n_sentences_in_chunk: int, overlap=0) -> list:
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    chunks = []
    n_sentences_in_chunk = n_sentences_in_chunk - overlap
    remainder = 0
    for i in range(overlap, len(sents), n_sentences_in_chunk):
        if remainder == n_sentences_in_chunk:
            remainder = 0
        chunk = sents[i-overlap-remainder:i + n_sentences_in_chunk-remainder]
        remainder = 0
        while len(" ".join(chunk)) / 4 > CONTEXT_WINDOW:
            remainder += 1
            chunk = chunk[:-remainder]
            
        chunk = " ".join(chunk)
        chunks.append(chunk)
    return chunks

In [209]:
dict_data = data.to_dict("records")  

for elem in tqdm(dict_data):
    elem["chunks"] = split_text_into_chunks(elem["Text"], n_sentences_in_chunk, overlap=1)

100%|██████████| 1391/1391 [00:03<00:00, 402.28it/s]


#### Example of chunking

In [210]:
for chunk in dict_data[0]["chunks"]:
    print(len(chunk) / 4)
    print(chunk)
    print("\n\n")

285.0
1. Introduction of Word2vec Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and human might cluster in one corner, while yellow, red and blue cluster together in another. There are two main training algorithms for word2vec, one is the continuous bag of words(CBOW), another is called skip-gram. The major difference between these two methods is that CBOW is using context to predict a target word while skip-gram is using a word to predict a target context. Generally, the skip-gram method can have a better performance compared with CBOW method, for it can captur

#### Creating a dataset with chunks

In [211]:
import re

chunks = pd.DataFrame(columns=["Title", "Chunk", "Chunk_length", "Chunk_words_num", "Chunk_sentences_num", "Token_count"])

for d in dict_data:
    for chunk in d["chunks"]:
        chunk = chunk.strip()
        chunk = re.sub(r"\s+", " ", chunk)
        row = pd.DataFrame.from_dict(
            {
                "Title": d["Title"],
                "Chunk": chunk,
                "Chunk_length": len(chunk),
                "Chunk_words_num": len(chunk.split()),
                "Chunk_sentences_num": len(list(nlp(chunk).sents)),
                "Token_count": len(chunk) / 4
            },
            orient="index"
        )
        row = row.T
        chunks = pd.concat([chunks, row], axis=0)
        
numeric = ["Chunk_length", "Chunk_words_num", "Chunk_sentences_num", "Token_count"]
chunks[numeric] = chunks[numeric].astype(float)

chunks.head()

,Title,Chunk,Chunk_length,Chunk_words_num,Chunk_sentences_num,Token_count
0,A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model,"1. Introduction of Word2vec Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space. For instance, the words women, men, and human might cluster in one corner, while yellow, red and blue cluster together in another. There are two main training algorithms for word2vec, one is the continuous bag of words(CBOW), another is called skip-gram. The major difference between these two methods is that CBOW is using context to predict a target word while skip-gram is using a word to predict a target context. Generally, the skip-gram method can have a better performance compared with CBOW method, for it can capture two semantics for a single word. For instance, it will have two vector representations for Apple, one for the company and another for the fruit.",1140.0,188.0,10.0,285.0
0,A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model,"For instance, it will have two vector representations for Apple, one for the company and another for the fruit. For more details about the word2vec algorithm, please check here. 2. Gensim Python Library Introduction Gensim is an open source python library for natural language processing and it was developed and is maintained by the Czech natural language processing researcher Radim Řehůřek. Gensim library will enable us to develop word embeddings by training our own word2vec models on a custom corpus either with CBOW of skip-grams algorithms. At first, we need to install the genism package. Gensim runs on Linux, Windows and Mac OS X, and should run on any other platform that supports Python 2.7+ and NumPy. Gensim depends on the following software: Python >= 2.7 (tested with versions 2.7, 3.5 and 3.6) >= 2.7 (tested with versions 2.7, 3.5 and 3.6) NumPy >= 1.11.3 >= 1.11.3 SciPy >= 0.18.1 >= 0.18.1 Six >= 1.5.0 >= 1.5.0 smart_open >= 1.2.1 There are two ways for installation. We could run the following code in our terminal to install genism package. pip install --upgrade gensim Or, alternatively for Conda environments: conda install -c conda-forge gensim 3.",1174.0,194.0,10.0,293.5
0,A Beginner’s Guide to Word Embedding with Gensim Word2Vec Model,"pip install --upgrade gensim Or, alternatively for Conda environments: conda install -c conda-forge gensim 3. Implementation of word Embedding with Gensim Word2Vec Model In this tutorial, I will show how to generate word embedding with genism using a concrete example. The dataset I used for this tutorial is from Kaggle Dataset. This vehicle dataset includes features such as make, model, year, engine, and other properties of the car. We will use these features to generate the word embeddings for each make model and then compare the similarities between different make model. The full python tutorial can be found here. >>> df = pd.read_csv('data.csv') >>> df.head() 3.1 Data Preprocessing: Since the purpose of this tutorial is to learn how to generate word embeddings using genism library, we will not do the EDA and feature selection for the word2vec model for the sake of simplicity. Genism word2vec requires that a format of ‘list of lists’ for training where every document is contained in a list and every list contains lists of tokens of that document. At first, we need to generate a format of ‘list of lists’ for training the make model word embedding. To be more specific, each make model is contained in a list and every list contains lists of features of that make model.",1288.0,215.0,10.0,322.0
0,A Beginner’s Guide to Word Embedding with G

In [212]:
chunks.sample(1)

,Title,Chunk,Chunk_length,Chunk_words_num,Chunk_sentences_num,Token_count
0,Step-by-Step Guide to Creating R and Python Libraries (in JupyterLab),"CREATING LIBRARIES IN R R is one of the “big 2” languages of machine learning. At the time of this writing it has well-over 10,000 libraries. Going to Available CRAN Packages By Date of Publication and running… document.getElementsByTagName('tr').length …in the browser console gives me 13858. Minus the header and final row this gives 13856 packages. Needless to say R is not in need of variety. With strong community support and a concise (if not intuitive) language, R sits comfortably at the top of statistical languages worth learning. The most well-known treatise on creating R packages is Hadley Wickam’s book R Packages. Its contents are available for free online. For a deeper dive on topic I recommend looking there. We will use Hadley’s devtools package to abstract away the tedious tasks involved in creating packages.",830.0,134.0,10.0,207.5


In [213]:
chunks.describe().astype(float).round(2)

,Chunk_length,Chunk_words_num,Chunk_sentences_num,Token_count
count,7334.00,7334.00,7334.00,7334.00
mean,1052.25,173.26,8.80,263.06
std,331.35,54.58,2.16,82.84
min,0.00,0.00,0.00,0.00
25%,865.00,144.00,9.00,216.25
50%,1100.00,182.00,10.00,275.00
75%,1313.00,213.00,10.00,328.25
max,1536.00,291.00,10.00,384.00


In [214]:
chunks = chunks[chunks["Token_count"] > 0]

In [215]:
chunks.describe()

,Chunk_length,Chunk_words_num,Chunk_sentences_num,Token_count
count,7292.000000,7292.000000,7292.000000,7292.000000
mean,1058.311437,174.255623,8.854361,264.577859
std,322.506653,53.123813,2.061306,80.626663
min,9.000000,2.000000,1.000000,2.250000
25%,871.000000,145.000000,9.000000,217.750000
50%,1101.000000,182.000000,10.000000,275.250000
75%,1314.000000,213.250000,10.000000,328.500000
max,1536.000000,291.000000,10.000000,384.000000


#### Printing some chunks with low token count

In [216]:
MIN_TOKENS = 50

for ch in chunks[chunks["Token_count"] < 50].sample(5)["Chunk"].values:
    print(ch)
    print("\n\n")



Or will it simply distract from more pressing problems here on Earth? Let us know your thoughts below…



GitHub is an online platform based on Git. What do you need to get started with GitHub? Installation



8. Horizontal Flip The generator will generate images, which on a random basis, will be horizontally flipped. 9. Vertical Flip Instead of flipping horizontally, we can also apply a vertical flip.



I’m currently seeking a full time data science position! More at lukepersola.com.



Informally, this is called a Swiss roll, a canonical problem in the field of non-linear dimensionality reduction. Some popular manifold learning methods are,





As we can see most of these short sentences come from the end of the articles. We can safely remove them from the dataset as they bring little information to the table.

In [217]:
print(chunks.shape)
chunks = chunks[chunks["Token_count"] > MIN_TOKENS]
print(chunks.shape)

(7292, 6)
(7185, 6)


#### Embedding the chunks